In [27]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [28]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import pandas as pd

In [29]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

In [30]:
# Download the multi-mnist dataset from Kaggle and save it in /dataset/multi_mnist directory in the folder containing this notebook.
# Then run this cell.

train_df = pd.read_csv(os.getcwd()+"/dataset/multi_mnist/train.csv")
test_df = pd.read_csv(os.getcwd()+"/dataset/multi_mnist/test.csv")
train_images = os.getcwd() + "/dataset/multi_mnist/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/dataset/multi_mnist/test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [31]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    # image.set_shape((64, 64, 1))
    # label[0].set_shape([])
    # label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

In [33]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [34]:
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=(3,3),
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)
output1 = layers.Dense(10, activation="softmax", name="first_num")(x)
output2 = layers.Dense(10, activation="softmax", name="second_num")(x)
model = keras.Model(inputs=inputs, outputs=[output1, output2])

model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=[
      keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    ],
    metrics=["accuracy"],
)

In [35]:
model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)

Epoch 1/5
1000/1000 - 270s - loss: 2.0895 - first_num_loss: 1.0180 - second_num_loss: 0.9972 - first_num_accuracy: 0.6334 - second_num_accuracy: 0.6412
Epoch 2/5
1000/1000 - 65s - loss: 0.6353 - first_num_loss: 0.2923 - second_num_loss: 0.2855 - first_num_accuracy: 0.9014 - second_num_accuracy: 0.9034
Epoch 3/5
1000/1000 - 65s - loss: 0.4189 - first_num_loss: 0.1853 - second_num_loss: 0.1819 - first_num_accuracy: 0.9390 - second_num_accuracy: 0.9384
Epoch 4/5
1000/1000 - 67s - loss: 0.3280 - first_num_loss: 0.1422 - second_num_loss: 0.1368 - first_num_accuracy: 0.9541 - second_num_accuracy: 0.9535
Epoch 5/5
1000/1000 - 66s - loss: 0.2780 - first_num_loss: 0.1186 - second_num_loss: 0.1126 - first_num_accuracy: 0.9621 - second_num_accuracy: 0.9622
313/313 - 19s - loss: 1.3573 - first_num_loss: 0.3791 - second_num_loss: 0.9316 - first_num_accuracy: 0.8978 - second_num_accuracy: 0.7585


[1.3572583198547363,
 0.3790723979473114,
 0.9315716624259949,
 0.897849977016449,
 0.7584999799728394]